In [1]:
%load_ext nb_black
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import seaborn as sns

%matplotlib inline

<IPython.core.display.Javascript object>

In [2]:
# Load the houseprices data from Thinkful's database.
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "houseprices"
table_name = "houseprices"

db_url = f"postgresql://{postgres_user}:{postgres_pw}@{postgres_host}:{postgres_port}/{postgres_db}"

engine = create_engine(db_url)

prices_df = pd.read_sql_query(f"SELECT * FROM {table_name};", con=engine)

# close the db connetion
engine.dispose()

<IPython.core.display.Javascript object>

In [3]:
prices_df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


<IPython.core.display.Javascript object>

#### Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.

In [4]:
prices_df2 = prices_df.copy()
cols_to_drop = [
    "lotfrontage",
    "fireplacequ",
    "fence",
    "alley",
    "miscfeature",
    "poolqc",
    "id",
]
prices_df2.drop(columns=cols_to_drop, inplace=True)
print(
    (prices_df2.isnull().sum() / prices_df2.isnull().count())
    .sort_values(ascending=False)
    .head(14)
)
prices_df2.head()

garagetype      0.055479
garageyrblt     0.055479
garagefinish    0.055479
garagequal      0.055479
garagecond      0.055479
bsmtfintype2    0.026027
bsmtexposure    0.026027
bsmtfintype1    0.025342
bsmtcond        0.025342
bsmtqual        0.025342
masvnrtype      0.005479
masvnrarea      0.005479
electrical      0.000685
yearremodadd    0.000000
dtype: float64


,mssubclass,mszoning,lotarea,street,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,...,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,...,0,0,0,0,0,12,2008,WD,Normal,250000


<IPython.core.display.Javascript object>

In [5]:
cols_with_nulls = (
    (prices_df2.isnull().sum() / prices_df2.isnull().count())
    .sort_values(ascending=False)
    .head(13)
    .index
)
cols_with_nulls

Index(['garagetype', 'garageyrblt', 'garagefinish', 'garagequal', 'garagecond',
       'bsmtfintype2', 'bsmtexposure', 'bsmtfintype1', 'bsmtcond', 'bsmtqual',
       'masvnrtype', 'masvnrarea', 'electrical'],
      dtype='object')

<IPython.core.display.Javascript object>

In [6]:
prices_df2.dropna(inplace=True)
(prices_df2.isnull().sum() / prices_df2.isnull().count()).sort_values(
    ascending=False
).head(12)

saleprice       0.0
roofstyle       0.0
exterior1st     0.0
exterior2nd     0.0
masvnrtype      0.0
masvnrarea      0.0
exterqual       0.0
extercond       0.0
foundation      0.0
bsmtqual        0.0
bsmtcond        0.0
bsmtexposure    0.0
dtype: float64

<IPython.core.display.Javascript object>

In [7]:
y = prices_df2["saleprice"]
X = prices_df2.select_dtypes(include="number")
X = X.drop("saleprice", axis=1)

<IPython.core.display.Javascript object>

In [8]:
import statsmodels.api as sm

X_constant = sm.add_constant(X)

results = sm.OLS(y, X_constant).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     166.5
Date:                Tue, 20 Oct 2020   Prob (F-statistic):               0.00
Time:                        20:36:27   Log-Likelihood:                -15881.
No. Observations:                1338   AIC:                         3.183e+04
Df Residuals:                    1304   BIC:                         3.201e+04
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          4.378e+05   1.49e+06      0.295      0.768   -2.48e+06    3.35e+06
mssubclass     -165.1681     28.434     -5.809      0.000    -220.949    -109.388
lotarea           0.3958      0.102      3.872      0.000       0.195       0.596
overallqual    1.855e+04   1275.557     14.540      0.000     1.6e+04     2.1e+04
overallcond    5534.6166   1144.242      4.837      0.000    3289.861    7779.373
yearbuilt       343.9801     78.751      4.368      0.000     189.488     498.472
yearremodadd    149.8357     75.673      1.980      0.048       1.382     298.289
masvnrarea       27.9619      6.108      4.578      0.000      15.979      39.945
bsmtfinsf1       10.6733      2.742      3.893      0.000       5.295      16.052
bsmtfinsf2        2.0032      4.750      0.422      0.673      -7.316      11.322
bsmtunfsf         0.6859      2.753      0.249      0.803      -4.715       6.086
totalbsmtsf      13.3624      4.716      2.833      0.005       4.110      22.614
firstflrsf       10.9442      7.784      1.406      0.160      -4.327      26.215
secondflrsf      18.4978      6.795      2.722      0.007       5.168      31.827
lowqualfinsf      1.1739     18.506      0.063      0.949     -35.130      37.478
grlivarea        30.6158      6.917      4.426      0.000      17.046      44.186
bsmtfullbath   8479.4957   2772.137      3.059      0.002    3041.159    1.39e+04
bsmthalfbath   2025.7138   4268.944      0.475      0.635   -6349.035    1.04e+04
fullbath       1104.6852   3070.206      0.360      0.719   -4918.398    7127.768
halfbath      -3942.0257   2847.014     -1.385      0.166   -9527.255    1643.203
bedroomabvgr  -9649.4986   1874.168     -5.149      0.000   -1.33e+04   -5972.785
kitchenabvgr  -2.585e+04   6627.661     -3.900      0.000   -3.89e+04   -1.28e+04
totrmsabvgrd   5609.9995   1301.906      4.309      0.000    3055.940    8164.059
fireplaces     3781.0681   1851.982      2.042      0.041     147.878    7414.258
garageyrblt     -49.3302     81.631     -0.604      0.546    -209.473     110.812
garagecars     1.522e+04   3077.207      4.947      0.000    9187.282    2.13e+04
garagearea        4.8244     10.396      0.464      0.643     -15.570      25.219
wooddecksf       23.8201      8.243      2.890      0.004       7.648      39.992
openporchsf      -6.7977     16.319     -0.417      0.677     -38.812      25.217
enclosedporch    11.1305     17.855      0.623      0.533     -23.897      46.158
threessnporch    23.4798     32.042      0.733      0.464     -39.381      86.340
screenporch      54.7444     17.384      3.149      0.002      20.640      88.849
poolarea        -37.1912     23.894     -1.556      0.120     -84.067       9.685
miscval          -0.6975      1.908     -0.365      0.715      -4.441       3.046
mosold          -42.0635    363.835     -0.116   

<IPython.core.display.Javascript object>

In [9]:
significant_cols = list(results.pvalues.loc[results.pvalues < 0.05].index)
significant_cols

['mssubclass',
 'lotarea',
 'overallqual',
 'overallcond',
 'yearbuilt',
 'yearremodadd',
 'masvnrarea',
 'bsmtfinsf1',
 'totalbsmtsf',
 'secondflrsf',
 'grlivarea',
 'bsmtfullbath',
 'bedroomabvgr',
 'kitchenabvgr',
 'totrmsabvgrd',
 'fireplaces',
 'garagecars',
 'wooddecksf',
 'screenporch']

<IPython.core.display.Javascript object>

In [10]:
X = prices_df2[significant_cols]

X_constant = sm.add_constant(X)

results = sm.OLS(y, X_constant).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.807
Model:                            OLS   Adj. R-squared:                  0.804
Method:                 Least Squares   F-statistic:                     290.1
Date:                Tue, 20 Oct 2020   Prob (F-statistic):               0.00
Time:                        20:36:27   Log-Likelihood:                -15885.
No. Observations:                1338   AIC:                         3.181e+04
Df Residuals:                    1318   BIC:                         3.191e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -9.402e+05   1.26e+05     -7.434      0.000   -1.19e+06   -6.92e+05
mssubclass    -165.8425     27.937     -5.936      0.000    -220.649    -111.036
lotarea          0.4075      0.101      4.027      0.000       0.209       0.606
overallqual   1.874e+04   1250.728     14.985      0.000    1.63e+04    2.12e+04
overallcond   5497.1824   1118.704      4.914      0.000    3302.548    7691.817
yearbuilt      294.8919     58.312      5.057      0.000     180.498     409.286
yearremodadd   146.3262     72.440      2.020      0.044       4.215     288.437
masvnrarea      28.3636      5.999      4.728      0.000      16.594      40.133
bsmtfinsf1       9.7472      3.142      3.102      0.002       3.583      15.911
totalbsmtsf     14.3211      5.906      2.425      0.015       2.736      25.907
secondflrsf      4.2236      6.031      0.700      0.484      -7.608      16.056
grlivarea       41.6965      6.631      6.288      0.000      28.688      54.705
bsmtfullbath  8106.1567   2476.558      3.273      0.001    3247.730     1.3e+04
bedroomabvgr -9314.2947   1824.482     -5.105      0.000   -1.29e+04   -5735.088
kitchenabvgr -2.501e+04   6498.714     -3.849      0.000   -3.78e+04   -1.23e+04
totrmsabvgrd  5548.5677   1285.878      4.315      0.000    3025.976    8071.159
fireplaces    3642.0051   1802.004      2.021      0.043     106.895    7177.115
garagecars    1.623e+04   2118.335      7.662      0.000    1.21e+04    2.04e+04
wooddecksf      22.6418      8.085      2.800      0.005       6.780      38.503
screenporch     49.3255     17.001      2.901      0.004      15.974      82.677
==============================================================================
Omnibus:                      712.419   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           129732.704
Skew:                          -1.364   Prob(JB):                         0.00
Kurtosis:                      51.162   Cond. No.                     2.00e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large,  2e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<IPython.core.display.Javascript object>

In [17]:
significant_cols = list(results.pvalues.loc[results.pvalues < 0.025].index)
significant_cols.remove("const")
significant_cols

['mssubclass',
 'lotarea',
 'overallqual',
 'overallcond',
 'yearbuilt',
 'masvnrarea',
 'grlivarea',
 'bsmtfullbath',
 'bedroomabvgr',
 'kitchenabvgr',
 'totrmsabvgrd',
 'garagecars']

<IPython.core.display.Javascript object>

In [18]:
X = prices_df2[significant_cols]

X_constant = sm.add_constant(X)

results = sm.OLS(y, X_constant).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.799
Model:                            OLS   Adj. R-squared:                  0.797
Method:                 Least Squares   F-statistic:                     439.4
Date:                Tue, 20 Oct 2020   Prob (F-statistic):               0.00
Time:                        20:37:46   Log-Likelihood:                -15912.
No. Observations:                1338   AIC:                         3.185e+04
Df Residuals:                    1325   BIC:                         3.192e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -8.442e+05   9.62e+04     -8.778      0.000   -1.03e+06   -6.55e+05
mssubclass    -214.0235     25.183     -8.499      0.000    -263.426    -164.621
lotarea          0.5238      0.100      5.214      0.000       0.327       0.721
overallqual   1.989e+04   1219.946     16.305      0.000    1.75e+04    2.23e+04
overallcond   6451.2958   1015.050      6.356      0.000    4460.016    8442.576
yearbuilt      393.0094     48.568      8.092      0.000     297.730     488.289
masvnrarea      33.8219      5.942      5.692      0.000      22.165      45.479
grlivarea       56.2658      3.943     14.269      0.000      48.530      64.001
bsmtfullbath  1.545e+04   1982.604      7.793      0.000    1.16e+04    1.93e+04
bedroomabvgr -1.158e+04   1790.209     -6.469      0.000   -1.51e+04   -8069.781
kitchenabvgr  -2.28e+04   6346.377     -3.593      0.000   -3.53e+04   -1.04e+04
totrmsabvgrd  4607.7249   1294.289      3.560      0.000    2068.645    7146.805
garagecars    1.686e+04   2127.591      7.923      0.000    1.27e+04     2.1e+04
==============================================================================
Omnibus:                      479.152   Durbin-Watson:                   1.953
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            64931.949
Skew:                          -0.562   Prob(JB):                         0.00
Kurtosis:                      37.109   Cond. No.                     1.49e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.49e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<IPython.core.display.Javascript object>